In [1]:
from huggingface_hub import notebook_login

notebook_login(write_permission=True)

In [2]:
from datasets import load_dataset, Audio

minds = load_dataset("azizbekphd/ikhlas_recitations", split="train")

In [3]:
minds = minds.train_test_split(test_size=0.2)

In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 1202
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 301
    })
})

In [5]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-large-960h")

In [6]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [7]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"] = len(batch["input_values"][0])
    return batch

In [8]:
encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/1202 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/301 [00:00<?, ? examples/s]

In [9]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [10]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

In [11]:
import evaluate

wer = evaluate.load("wer")

In [12]:
import numpy as np


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

In [ ]:
training_args = TrainingArguments(
    output_dir="wav2vec2-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    push_to_hub=True,
    fp16=False,
    use_mps_device=False,
    use_cpu=True,
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    processing_class=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/azizbekphd/Projects/ikhlas_mdd/venv/lib/python3.10/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 11.2299, 'grad_norm': 428.3405456542969, 'learning_rate': 4.866666666666667e-05, 'epoch': 0.13}
{'loss': 2.4596, 'grad_norm': 149.62319946289062, 'learning_rate': 4.7333333333333336e-05, 'epoch': 0.26}
